In [8]:
import os
import numpy as np

FILES_DIR = 'saliency_experiment_outputs/'

# list all files in the directory
folders = os.listdir(FILES_DIR)
for folder in folders:
    # open numpy_saliency_maps folder
    folder_path = os.path.join(FILES_DIR, folder, 'numpy_saliency_maps')
    
    # Check if the specific numpy_saliency_maps folder exists
    if not os.path.isdir(folder_path):
        print(f"Directory {folder_path} not found, skipping...")
        continue
        
    batch_maps_path = os.path.join(folder_path, 'batch_maps.npy')
    
    # Skip if batch_maps.npy already exists
    if os.path.exists(batch_maps_path):
        print(f"Batch maps already exist for {folder} at {batch_maps_path}, skipping...")
        continue
        
    batch_maps = np.array([])
    
    # Get file list and sort it
    try:
        files_in_folder = os.listdir(folder_path)
        files_in_folder.sort() # Sorts alphabetically. Use a natural sort key if needed for e.g. map1, map10, map2
    except FileNotFoundError:
        print(f"Error listing files in {folder_path}, skipping folder {folder}...")
        continue

    if not files_in_folder:
        print(f"No files found in {folder_path} for folder {folder}, skipping saving batch_maps.")
        continue # Skip saving if no files to process

    for file_name in files_in_folder:
        # Skip the target batch file itself
        if file_name == 'batch_maps.npy':
            print(f"Skipping '{file_name}' as it is the target batch file.")
            continue

        # Ensure we are only processing .npy files
        if not file_name.endswith('.npy'):
            # You might want to print a message here if you expect only .npy files
            # print(f"Skipping non-npy file: {file_name} in {folder_path}")
            continue
            
        file_path = os.path.join(folder_path, file_name)
        
        try:
            saliency_map = np.load(file_path)
        except Exception as e:
            print(f"Error loading {file_path}: {e}. Skipping this file.")
            continue

        # Check the shape of the loaded saliency map
        if len(saliency_map.shape) == 2:
            # If it's a 2D array (H, W), add a new axis to make it 3D (1, H, W)
            saliency_map = saliency_map[np.newaxis, :, :]
        elif len(saliency_map.shape) == 3 and saliency_map.shape[0] == 1:
            # It's already in (1, H, W) format, which is fine
            pass
        else:
            print(f"Warning: Unexpected shape for {file_name}: {saliency_map.shape}. Stopping processing for folder {folder}.")
            batch_maps = np.array([]) # Clear any partially processed maps for this folder
            break # Stop processing this folder
            
        # if batch_maps is empty, set it to saliency_map
        if batch_maps.size == 0:
            batch_maps = saliency_map
        else:
            # Make sure dimensions match before concatenating (H, W must match)
            if batch_maps.shape[1:] != saliency_map.shape[1:]:
                print(f"Warning: Shape mismatch - batch_maps: {batch_maps.shape}, saliency_map: {saliency_map.shape} for file {file_name}. Skipping this file.")
                continue
            # concatenate the saliency map to the batch_maps array
            try:
                batch_maps = np.concatenate((batch_maps, saliency_map), axis=0)
            except ValueError as e:
                print(f"Error concatenating {file_name} with shape {saliency_map.shape} to batch_maps with shape {batch_maps.shape}: {e}. Skipping this file.")
                continue
    
    # Only save if batch_maps has content and the loop wasn't broken prematurely due to an unrecoverable error
    if batch_maps.size > 0:
        try:
            np.save(batch_maps_path, batch_maps)
            print(f'Batch maps shape for {folder}: {batch_maps.shape}, saved to {batch_maps_path}')
        except Exception as e:
            print(f"Error saving batch_maps to {batch_maps_path} for folder {folder}: {e}")
    elif not files_in_folder: 
        pass # Already handled: "No files found..."
    else:
        print(f"No valid saliency maps processed for folder {folder}. Not saving batch_maps.npy.")

Batch maps shape for Pruned_Model: (13, 224, 224), saved to saliency_experiment_outputs/Pruned_Model/numpy_saliency_maps/batch_maps.npy
Batch maps shape for Baseline_Model: (13, 224, 224), saved to saliency_experiment_outputs/Baseline_Model/numpy_saliency_maps/batch_maps.npy
